# Read this first 

* each comment and text line added here is important and relevant , dont skim through just the code
* if the comment asks you to revisit some concepts/modules , do so
* this is not complete code on its own , you need to fill in your own code to eventually start building a model

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [2]:
datafile_train="/Users/lalitsachan/Dropbox/March onwards/PDS V2/Projects/P3/counterfeit_train.csv"
datafile_test="/Users/lalitsachan/Dropbox/March onwards/PDS V2/Projects/P3/counterfeit_test.csv"
bd_train=pd.read_csv(datafile_train)
bd_test=pd.read_csv(datafile_test)

In [3]:
bd_train.isnull().sum()

Medicine_ID               0
Counterfeit_Weight     1166
DistArea_ID               0
Active_Since              0
Medicine_MRP              0
Medicine_Type             0
SidEffect_Level           0
Availability_rating       0
Area_Type                 0
Area_City_Type            0
Area_dist_level           0
Counterfeit_Sales         0
dtype: int64

In [4]:
bd_train['Medicine_ID'].nunique()

1557

In [5]:
bd_train.shape

(6818, 12)

In [6]:
# in this script we are going to ignore two column Medicing_ID, Counterfeit_Weight
# for improving your results , you should try imputing values for Counterfeit_Weight
# see if you see any pattern between Medcine_ID and Counterfeit_Weight and try to make use of it 



In [7]:
for col in [ 'Medicine_ID', 'Counterfeit_Weight']:
    bd_train.drop(col,1,inplace=True)
    bd_test.drop(col,1,inplace=True)

In [8]:
# for the remaining cat vars we'll create dummies 
# you can try creating dummies with freq cutoff and see if that improves results 

In [9]:
for col in ['Medicine_Type','SidEffect_Level','Area_Type','Area_City_Type','Area_dist_level',"DistArea_ID"]:  
    temp=pd.get_dummies(bd_train[col],prefix=col,drop_first=True)
    bd_train=pd.concat([temp,bd_train],1)
    bd_train.drop([col],1,inplace=True)
       
    temp=pd.get_dummies(bd_test[col],prefix=col,drop_first=True)
    bd_test=pd.concat([temp,bd_test],1)
    bd_test.drop([col],1,inplace=True)

In [10]:
bd_train.shape

(6818, 37)

In [11]:
target='Counterfeit_Sales'

In [12]:
x_train=bd_train.drop(target,1)
y_train=bd_train[target]

In [13]:
# we are using Lasso model here , you can chose more complex algos for 
# better performance 

In [14]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [15]:
params={'alpha':np.linspace(0.1,100,50)}

In [16]:
model=Lasso(fit_intercept=True)

In [17]:
grid_search=GridSearchCV(model,cv=10,param_grid=params,n_jobs=-1,verbose=10,
                         scoring='neg_mean_absolute_error')
# it will better idea to use randomised search for more complex algos

In [18]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1925s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  75 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1422s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 217 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 285 tasks      | elapsed:    4.2s
[Parallel(n_j

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': array([  0.1    ,   2.13878,   4.17755,   6.21633,   8.2551 ,  10.29388,
        12.33265,  14.37143,  16.4102 ,  18.44898,  20.48776,  22.52653,
        24.56531,  26.60408,  28.64286,  30.68163,  32.72041,  34.75918,
        36.79796,  38.83673,  40.87551,  42.91429,  44.95306...     85.72857,  87.76735,  89.80612,  91.8449 ,  93.88367,  95.92245,
        97.96122, 100.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=10)

In [19]:
submissions=pd.DataFrame({target:grid_search.predict(bd_test)})

In [20]:
submissions.to_csv('sample_submission.csv',index=False)